In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import statsmodels.discrete.discrete_model as sm
import statsmodels.formula.api as smf

In [160]:
dfr=pd.read_csv('tempSiteRatings.csv')
dfd=pd.read_csv('tempUrlMetrics.csv')

In [161]:
#print(dfd['slno'].values==dfr['id'].values)
dfd['rating']=dfr['rating']
dfd.to_csv('dataClf.csv')

In [162]:
def changeColName(data):
    try:
        columns=['word_count','body_text_percent','emphasized_text_percent','text_position_changes','text_cluster_count','link_count','page_size','graphic_percent','graphic_count','colour_count','font_count','colourfulness','visual_complexity','rating']
        data.columns=columns
    except:
        columns=['word_count','body_text_percent','emphasized_text_percent','text_position_changes','text_cluster_count','link_count','page_size','graphic_percent','graphic_count','colour_count','font_count','colourfulness','visual_complexity']
        data.columns=columns
    return data

In [163]:
def dropMetrics(data):
    try:
        data.drop(['id','url'],1, inplace=True)#,'p1','p2','p4','p6','p7','p10','p11','p12','p13'
    except:
        data.drop(['slno','url'],1, inplace=True)
    return data

In [164]:
def stdNormal(data,mean,std):
    x = data.values #returns a numpy array
    columns=data.columns
    x_scaled=(x-mean)/std
    data=pd.DataFrame(x_scaled,columns=columns)
    return data

In [165]:
def catCol(data,col):
    data[col+'_high']=np.zeros(data[col].shape)
    data[col+'_avg']=np.ones(data[col].shape)
    data[col+'_low']=np.zeros(data[col].shape)

    data[col+'_high'][np.where(data[col]>.431)[0]]=1.0
    data[col+'_avg'][np.where(data[col]>.431)[0]]=0.0
    data[col+'_avg'][np.where(data[col]<-.431)[0]]=0.0
    data[col+'_low'][np.where(data[col]<-.431)[0]]=1.0
    data.drop([col],1,inplace=True)
    return data

In [166]:
def catColPdf(data,col):
    import scipy.stats
    
    mean_high=np.mean(data[col][np.where(data[col]>.431)[0]])
    std_high=np.std(data[col][np.where(data[col]>.431)[0]])


    mean_avg=np.mean(data[col][np.where(abs(data[col])<.431)[0]])
    std_avg=np.std(data[col][np.where(abs(data[col])<.431)[0]])

    mean_low=np.mean(data[col][np.where(data[col]<-.431)[0]])
    std_low=np.std(data[col][np.where(data[col]<-.431)[0]])

    data[col+'_high']=scipy.stats.norm(mean_high,std_high).pdf(data[col].values)
    data[col+'_avg']=scipy.stats.norm(mean_avg,std_avg).pdf(data[col].values)
    data[col+'_low']=scipy.stats.norm(mean_low,std_low).pdf(data[col].values)
    
    data.drop([col],1,inplace=True)
    
    return data


In [167]:
def preProcess(data,mean,std):
    try:
        rating=data['rating']
        data.drop(['rating'],1,inplace=True)
    except:
        pass
    data=stdNormal(data,mean,std)
    #data=catCol(data,'word_count')
    #data=catCol(data,'page_size') 
    #data=catCol(data,'colourfulness') 
    #data=catCol(data,'visual_complexity') 
    try:
        data['rating']=rating
    except:
        pass
    return data

In [168]:
dataClf=pd.read_csv('dataClf.csv')
dataClf.drop(['Unnamed: 0'],1, inplace=True)
dataClf=dropMetrics(dataClf)
dataClf=changeColName(dataClf)

goodInd=np.argpartition(dataClf['rating'].values,-25)[-25:]
badInd=np.argpartition(dataClf['rating'].values,25)[:25]
allInd=np.concatenate((goodInd,badInd),axis=0)

dataClf['rating'][badInd]=0
dataClf['rating'][goodInd]=1
data=pd.DataFrame(dataClf[dataClf.columns].values[allInd],columns=dataClf.columns)


/home/abhiavk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/abhiavk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [169]:
irrCol=['emphasized_text_percent','font_count']
data.drop(irrCol,1,inplace=True)
mean=np.mean(data.values[:,:-1].astype(float),axis=0)
std=np.std(data.values[:,:-1].astype(float),axis=0)


In [170]:
data=preProcess(data,mean,std)

In [171]:
np.std(data.values.astype(float),axis=0)

array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.5])

In [172]:
correlation = data.corr(method='pearson')
columns = correlation.nlargest(30,'rating').index
columns=columns.tolist()
columns.sort()
columns=pd.Index(columns)

In [173]:
Y=data['rating'].values
X_temp=data[columns]
X_temp.drop(['rating'],1, inplace=True)
X=X_temp.values
columns=columns.drop('rating')

In [174]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True,test_size=0.25, random_state=1)

In [175]:
columns

Index(['body_text_percent', 'colour_count', 'colourfulness', 'graphic_count',
       'graphic_percent', 'link_count', 'page_size', 'text_cluster_count',
       'text_position_changes', 'visual_complexity', 'word_count'],
      dtype='object')

In [176]:
col='rating ~ '
for i in columns:
    col+=i+' + '
col=col[:-3]
col

'rating ~ body_text_percent + colour_count + colourfulness + graphic_count + graphic_percent + link_count + page_size + text_cluster_count + text_position_changes + visual_complexity + word_count'

In [177]:
clf=sm.Logit(Y,X).fit()

Optimization terminated successfully.
         Current function value: 0.444651
         Iterations 7


In [178]:
clf=smf.logit(col,data=data).fit(maxiter=35000)

Optimization terminated successfully.
         Current function value: 0.434496
         Iterations 8


In [179]:
clf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 rating   No. Observations:                   50
Model:                          Logit   Df Residuals:                       38
Method:                           MLE   Df Model:                           11
Date:                Tue, 18 Jun 2019   Pseudo R-squ.:                  0.3732
Time:                        09:28:56   Log-Likelihood:                -21.725
converged:                       True   LL-Null:                       -34.657
                                        LLR p-value:                  0.006797
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.4853      0.502     -0.967      0.334      -1.469       0.498
body_text_percent         0.0170      0.389      0.044      0.965      -0.746       0.780
colour_count             -0.5036      0.555     -0.907      0.364      -1.592       0.585
colourfulness            -0.2538      0.411     -0.618      0.536      -1.058       0.551
graphic_count             2.1292      0.758      2.808      0.005       0.643       3.616
graphic_percent           0.1102      0.417      0.264      0.792      -0.707       0.928
link_count                2.0501      1.632      1.256      0.209      -1.149       5.249
page_size                -0.1026      0.313     -0.328      0.743      -0.716       0.511
text_cluster_count       -0.5937      0.512     -1.161      0.246      -1.596       0.409
text_position_changes    -1.4228      0.884     -1.609      0.108      -3.156       0.310
visual_complexity         0.1998      0.567      0.352      0.725      -0.912       1.312
word_count               -3.9405      1.701     -2.317      0.021      -7.274      -0.607
=========================================================================================
"""

In [180]:
columns

Index(['body_text_percent', 'colour_count', 'colourfulness', 'graphic_count',
       'graphic_percent', 'link_count', 'page_size', 'text_cluster_count',
       'text_position_changes', 'visual_complexity', 'word_count'],
      dtype='object')

In [181]:
text="""
Intercept	-0.4853	0.502	-0.967	0.334	-1.469	0.498
body_text_percent	0.0170	0.389	0.044	0.965	-0.746	0.780
colour_count	-0.5036	0.555	-0.907	0.364	-1.592	0.585
colourfulness	-0.2538	0.411	-0.618	0.536	-1.058	0.551
graphic_count	2.1292	0.758	2.808	0.005	0.643	3.616
graphic_percent	0.1102	0.417	0.264	0.792	-0.707	0.928
link_count	2.0501	1.632	1.256	0.209	-1.149	5.249
page_size	-0.1026	0.313	-0.328	0.743	-0.716	0.511
text_cluster_count	-0.5937	0.512	-1.161	0.246	-1.596	0.409
text_position_changes	-1.4228	0.884	-1.609	0.108	-3.156	0.310
visual_complexity	0.1998	0.567	0.352	0.725	-0.912	1.312
word_count	-3.9405	1.701	-2.317	0.021	-7.274	-0.607
"""
import re
text=re.sub('\t','&',text)
text=re.sub('\n','\\\\',text)
text

'\\Intercept&-0.4853&0.502&-0.967&0.334&-1.469&0.498\\body_text_percent&0.0170&0.389&0.044&0.965&-0.746&0.780\\colour_count&-0.5036&0.555&-0.907&0.364&-1.592&0.585\\colourfulness&-0.2538&0.411&-0.618&0.536&-1.058&0.551\\graphic_count&2.1292&0.758&2.808&0.005&0.643&3.616\\graphic_percent&0.1102&0.417&0.264&0.792&-0.707&0.928\\link_count&2.0501&1.632&1.256&0.209&-1.149&5.249\\page_size&-0.1026&0.313&-0.328&0.743&-0.716&0.511\\text_cluster_count&-0.5937&0.512&-1.161&0.246&-1.596&0.409\\text_position_changes&-1.4228&0.884&-1.609&0.108&-3.156&0.310\\visual_complexity&0.1998&0.567&0.352&0.725&-0.912&1.312\\word_count&-3.9405&1.701&-2.317&0.021&-7.274&-0.607\\'